In [0]:
# Data Lake name
storageAccountName = "datalakef30b94371f0fa0cb"

In [0]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze"))

path,name,size,modificationTime
dbfs:/mnt/datalakef30b94371f0fa0cb/bronze/application/,application/,0,0
dbfs:/mnt/datalakef30b94371f0fa0cb/bronze/bank_accounts/,bank_accounts/,0,0
dbfs:/mnt/datalakef30b94371f0fa0cb/bronze/cards/,cards/,0,0
dbfs:/mnt/datalakef30b94371f0fa0cb/bronze/loans/,loans/,0,0
dbfs:/mnt/datalakef30b94371f0fa0cb/bronze/transactions/,transactions/,0,0
dbfs:/mnt/datalakef30b94371f0fa0cb/bronze/users/,users/,0,0


In [0]:
# Lendo os arquivos delta e transformando em DataFrame
df_application   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/application")
df_bank_accounts   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/bank_accounts")
df_cards   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/cards")
df_loans   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/loans")
df_transactions   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/transactions")
df_users   = spark.read.format('delta').load(f"/mnt/{storageAccountName}/bronze/users")

In [0]:
# Adicionando data da criação e nome do arquivo de origem
from pyspark.sql.functions import current_timestamp, lit

df_application = df_application.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("application"))
df_bank_accounts = df_bank_accounts.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("bank_accounts"))
df_cards = df_cards.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("cards"))
df_loans = df_loans.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("loans"))
df_transactions = df_transactions.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("transactions"))
df_users = df_users.withColumn("data_hora_silver", current_timestamp()).withColumn("nome_arquivo", lit("users"))

In [0]:
#Fucao para renomear as colunas de cada delta table para maiuscula e ajusta o nome das colunas de acordo com o dicionario de dados
def renomear_colunas(diretorio):

    # Carregue o DataFrame a partir do delta
    df = spark.read.format('delta').load(diretorio)

    tabela = diretorio.split('/')[-2]

    # Renomeie todas as colunas para maiúsculas e faça a substituição do padrão "cd_" para "codigo_"
    for coluna in df.columns:
        novo_nome = coluna.upper()
        df = df.withColumnRenamed(coluna, novo_nome)
        df = df.drop("DATA_HORA_BRONZE")
        df = df.drop("NOME_ARQUIVO")
        df = df.withColumn("NOME_ARQUIVO_BRONZE", lit(tabela))
        df = df.withColumn("DATA_ARQUIVO_SILVER", current_timestamp())

    # Salve o DataFrame modificado de volta no mesmo local
    #df.display()
    df.write.format('delta').save(f"/mnt/{storageAccountName}/silver/{tabela}")

#Funcao que chama a funcao renomear colunas para todos os arquivos contidos no container
def renomear_arquivos_delta(diretorio):

    # Lista para armazenar os nomes dos arquivos delta
    nomes_arquivos_delta = []

    # Lista os arquivos no diretório
    arquivos = dbutils.fs.ls(diretorio)
    
    # Itera sobre os arquivos e armazena os nomes dos arquivos delta
    for arquivo in arquivos:
        nome_arquivo = arquivo.path
        # Remover as aspas e a barra (/) no final do nome do arquivo
        #nome_arquivo = nome_arquivo.rstrip('/')
        #nomes_arquivos_delta.append(nome_arquivo)
        renomear_colunas(nome_arquivo)
        #print(nome_arquivo)

    return nomes_arquivos_delta

In [0]:
# Executa a funcao para atualizar todos os dataframes
diretorio = f'/mnt/{storageAccountName}/bronze'

renomear_arquivos_delta(diretorio)

Out[16]: []

In [0]:
#Funcao auxiliar
import os

# Use a função para renomear as colunas para maiúsculo em um diretório específico
diretorio = f'/mnt/{storageAccountName}/bronze/'

arquivos = dbutils.fs.ls(diretorio)

# Imprime os nomes dos arquivos
print("Arquivos no diretório:")
for arquivo in arquivos:
    print(arquivo.name.rstrip('/'), arquivo.name.rstrip('/').upper())

Arquivos no diretório:
application APPLICATION
bank_accounts BANK_ACCOUNTS
cards CARDS
loans LOANS
transactions TRANSACTIONS
users USERS
